In [ ]:
import json, glob, gc
import pandas as pd
from pandas import json_normalize

import psycopg2   
from psycopg2 import Error
import geopandas as gpd
import numpy as np
import pandas as pd

from tqdm import tqdm
from shapely.geometry import LineString, Point
from shapely.wkt import dumps, loads

from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

In [ ]:
table_name = ""
database = ""
user = "postgres"
password = "postgres"
port = str(5432)
host = "localhost"

In [ ]:
engine = create_engine("postgresql://"+user+":"+password+"@"+host+":"+port+"/"+database)

def generate_geometry(x):
    return loads(dumps(Point(x['lng'], x['lat']), rounding_precision=10))

In [ ]:
csv_files = glob.glob('insta_hashtags*.csv')
failed = []

for insta_file in tqdm(range(len(csv_files))):
    
    try:
        df = pd.read_csv(csv_files[insta_file])
        df = df.drop_duplicates(subset=["caption_text"], keep='first')

        df['geometry'] = df.apply(generate_geometry, axis=1)
        df = df[df['geometry'].notna()]

        gpd_insta = gpd.GeoDataFrame(df, geometry='geometry')
        gpd_insta.set_crs(epsg=4326, inplace=True, allow_override=True)
        gpd_insta = gpd_insta.__round__(10)

        ## Create table in postgis and insert data
        gpd.GeoDataFrame.to_postgis(gpd_insta, name = table_name, con = engine, schema = 'public', if_exists = 'append')

        del gpd_insta, df
        gc.collect()
    except:
        failed.append(insta_file)
        print(len(failed))
        continue
